In [1]:
%load_ext autoreload
%autoreload 2

# Experiments with TEASER-S model and simulated feedback

In [2]:
from pathlib import Path
import numpy as np

import teaser.util as util
import teaser.evaluation as evaluation

from teaser.algorithm.teaser_s import TEASER_S

## Datasets

In [3]:
## Uncomment corresponding directory.
DATA_DIR = Path('../../data/ML20M/')
# DATA_DIR = Path('../../data/Amazon Video Games/')
# DATA_DIR = Path('../../data/Publiq/')

In [4]:
# Shouldn't need to change this info
INTERACTIONS_TRAIN_PATH = DATA_DIR / 'interactions.train.csv'
INTERACTIONS_VALIN_PATH = DATA_DIR / 'interactions.valin.csv'
INTERACTIONS_VALOUT_PATH = DATA_DIR / 'interactions.valout.csv'
INTERACTIONS_TESTIN_PATH = DATA_DIR / 'interactions.testin.csv'
INTERACTIONS_TESTOUT_PATH = DATA_DIR / 'interactions.testout.csv'

METADATA_PATH = DATA_DIR / 'features.csv'

ITEM_ID = 'itemId'
USER_ID = 'userId'

## Define algorithm and hyperparameter ranges

In [5]:
ALG = TEASER_S

# ML20M
L2_1 = 100
L2_2 = 100000000
RHO = 1000
DELTA = 0
MAX_ITER = 10

# AVG
# L2_1 = 500
# L2_2 = 2000
# RHO = 10000
# DELTA = 0
# MAX_ITER = 15

# Publiq
# L2_1 = 500
# L2_2 = 50000
# RHO = 1000
# DELTA = 0
# MAX_ITER = 10

HYPERPARAMS = {'l2_1': L2_1, 'l2_2': L2_2, 'rho': RHO, 'delta': DELTA,'max_iterations': MAX_ITER}
HYPERPARAMS

{'l2_1': 100, 'l2_2': 100000000, 'rho': 1000, 'delta': 0, 'max_iterations': 10}

## Parse data

In [6]:
S, tags = util.parse_metadata(METADATA_PATH, item_id=ITEM_ID)
n, t = S.shape  # amount of items and tags

X = util.parse_interactions(INTERACTIONS_TRAIN_PATH, item_id=ITEM_ID, user_id=USER_ID, shape_items=n)

Xtest_in = util.parse_interactions(INTERACTIONS_TESTIN_PATH, item_id=ITEM_ID, user_id=USER_ID, shape_items=n)
Xtest_out = util.parse_interactions(INTERACTIONS_TESTOUT_PATH, item_id=ITEM_ID, user_id=USER_ID, shape_items=n)

In [7]:
# # debug
# d_items = 1000
# S = S[:d_items,:]
# tags = tags[:d_items]
# X = X[:,:d_items]
# Xtest_in = Xtest_in[:,:d_items]
# Xtest_out = Xtest_out[:,:d_items]

## Evaluate model with optimal hyperparams on test set

In [8]:
# Precomput XTX and decomposition of XTX, since it's needed for all hyperparameters combinations and doesn't change
XTX = (X.T @ X).toarray()
p, U = np.linalg.eigh(XTX)
fit_params = {'XTX': XTX, 'p': p, 'U': U}

In [9]:
%%time
alg = ALG(**HYPERPARAMS)
alg.fit(X, S, **fit_params)

Decompose DTD


norm E 0.1681452872166037
diag norm: 0.11399075242549635
diag_diff: 2.363929507316824

norm E 0.16540632775221895
diag norm: 0.11217234396016726
diag_diff: 1.1916742464242325
rho * change E: 3.4051488172975164

norm E 0.16711708803567438
diag norm: 0.11322435441718634
diag_diff: 0.6035439873478129
rho * change E: 2.091047893033957

norm E 0.16789334082227445
diag norm: 0.11367490277421534
diag_diff: 0.31511990390257727
rho * change E: 0.9673670068445457

norm E 0.16833709341865988
diag norm: 0.11391009473811954
diag_diff: 0.18123724917007575
rho * change E: 0.5368068197486123

norm E 0.16859127593484713
diag norm: 0.11402335465006111
diag_diff: 0.12700848530665543
rho * change E: 0.2998128632097465

norm E 0.16874650694913243
diag norm: 0.11407308488877094
diag_diff: 0.1102396678461191
rho * change E: 0.1781282276208033

norm E 0.16882985118764474
diag norm: 0.11407500936128981
diag_diff: 0.10613862284895395
rho * change E: 0.09514390886370244

norm E 0.16888230339053556
diag norm: 0.1

In [10]:
print("Performance on test set with simulation")
for POS_FEEDBACK in [1, 2]:
    for STRENGTH in [3]:
        print("Pos feedback count:", POS_FEEDBACK, "with strength (out of 5):", STRENGTH)
        score = evaluation.simulate(alg, Xtest_in, Xtest_out, S, pos_feedback=POS_FEEDBACK, strength=STRENGTH, repeats=3)
    print()

Performance on test set with simulation
Pos feedback count: 1 with strength (out of 5): 3
Evaluating with 10000 users
Average Recall@20 0.128
Average Recall@100 0.267
Average nDCG@100 0.152

Pos feedback count: 2 with strength (out of 5): 3
Evaluating with 10000 users
Average Recall@20 0.151
Average Recall@100 0.305
Average nDCG@100 0.171

